In [2]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
# path to image directory
image_dir="E:/image_auto/train_Images"

# path to csv file where image id's and their corresponding labels are present
csv_dir="E:/image_auto/train.csv"

traindf=pd.read_csv(csv_dir,dtype=str)
traindf.head(5)

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [3]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.20)


train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=image_dir,
x_col="Image",
y_col="Class",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(80,80))


valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=image_dir,
x_col="Image",
y_col="Class",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(80,80))

Found 4787 validated image filenames belonging to 4 classes.
Found 1196 validated image filenames belonging to 4 classes.


In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'E:/weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (80, 80, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


from tensorflow.keras import optimizers
my_optimizer= optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = my_optimizer, 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 39, 39, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 39, 39, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [5]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history=model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 149 steps, validate for 37 steps
Epoch 1/10
149/149 [==============================] - 57s 385ms/step - loss: 1.1873 - acc: 0.5476 - val_loss: 1.7002 - val_acc: 0.5921
Epoch 2/10
149/149 [==============================] - 38s 252ms/step - loss: 0.7182 - acc: 0.7348 - val_loss: 2.0117 - val_acc: 0.5819
Epoch 3/10
149/149 [==============================] - 41s 276ms/step - loss: 0.4432 - acc: 0.8572 - val_loss: 2.4672 - val_acc: 0.5794
Epoch 4/10
149/149 [==============================] - 43s 288ms/step - loss: 0.2169 - acc: 0.9413 - val_loss: 2.6950 - val_acc: 0.5853
Epoch 5/10
149/149 [==============================] - 39s 264ms/step - loss: 0.1131 - acc: 0.9783 - val_loss: 3.4041 - val_acc: 0.5760
Epoch 6/10
149/149 [==============================] - 38s 258ms/step - loss: 0.1073 - acc: 0.9815 - val_loss: 3.5608 - val_acc: 0.5785
Epoch 7/10
149/149 [=

In [6]:
test_image_dir = "E:/image_auto/test_Images"
test_dir="E:/image_auto/test.csv"
testdf=pd.read_csv(test_dir,dtype=str)

test_datagen = ImageDataGenerator(rescale=1. / 255)


test_generator = test_datagen.flow_from_directory(
    directory=test_image_dir,
    target_size=(80,80),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)
        


Found 3219 images belonging to 1 classes.


In [7]:
pred = model.predict(test_generator)

In [8]:
testdf['Image']=testdf['Image'].astype(str)
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3219 entries, 0 to 3218
Data columns (total 1 columns):
Image    3219 non-null object
dtypes: object(1)
memory usage: 25.3+ KB


In [12]:
label = [np.argmax(i) for i in pred]
str_label= ['Food','misc','Attire','Decorationandsignage']
final_lab= ([str_label(i) for i in label])
# for i in label:
#     if i==0:
#         final_lab.append()


TypeError: 'list' object is not callable

In [24]:

submission = pd.DataFrame({
       "Image": testdf['Image'],
       "Class": final_lab
   })
submission.to_csv('E:/image_auto/s.csv', index=False)